<a href="https://colab.research.google.com/github/orlandokohjy/IPP-BSC/blob/main/IPP_BSC_v20211019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recovery script
Due to broken laptop, this is the only latest script I have for this project

In [ ]:
import os
import pandas as pd
import numpy as np


user_directory = '/Users/jiayikoh/Downloads/OneDrive_1_16-06-2021/'
insurance_file = 'Compliance BSC.xlsx'
investment_file = 'Jan - Mar 2020 CIS Report.xlsx'
product_file = 'Approved product and Bundled Poduct List 21 Jan 2020.xlsx'

#oldadvisor_file = input('File name of the Old Advisor information (including the format -> oldadvcheck.xlsx)')
    
os.chdir(user_directory)
compliance = pd.read_excel(insurance_file,header=[0]) 
compliance = pd.DataFrame(compliance)
cis = pd.read_excel(investment_file,header=[0]) 
cis = pd.DataFrame(cis)
pro = pd.read_excel(product_file, sheet_name = 'IPPFA App', header = [7])

##to manage insurance data
#Remove any data that contain blank in 'Advisor'
compliance.dropna(subset=['Advisor'], inplace = True)

#Remove duplicates except for two fields
compliance = compliance.drop_duplicates(subset=compliance.columns.difference(['Modified On','Case Modified On']),keep='first')

#Fill null values with blank string for data manipulation purpose
compliance[compliance['Plan Type'] == 'Term Life'][['Subject To BSC']].fillna('Yes',inplace=True)
compliance[['Accredited Investor','Subject To BSC']] = compliance[['Accredited Investor','Subject To BSC']].fillna('')


#create 'BSC' column and remove whitespace for all names columns in order to find whether is FAR own trade policy
compliance['Compliance BSC (Y/N)'] = ''

farreq1 = compliance['Subject To BSC'] == 'No (No Advice)'
farreq2 = compliance['Subject To BSC'] == 'Yes'

compliance['name_check_1'] = compliance.loc[:,'Advisor'].str.lower().str.strip().str.replace(' ','')

compliance['name_check_2'] = compliance.loc[:,'Contact Name'].str.lower().str.strip().str.replace(' ','')

compliance['name_check_3'] = compliance.loc[:,'All Advisors'].str.lower().str.strip().str.replace(' ','')

#to identify FAR own trade policy using name
def check_name(row):
    return row['name_check_1'] in row['name_check_2'] or row['name_check_1'] in row['name_check_3'] or row['name_check_2'] in row['name_check_1'] or row['name_check_2'] in row['name_check_3'] or row['name_check_3'] in row['name_check_1'] or row['name_check_3'] in row['name_check_2']

compliance.loc[compliance.apply(check_name, axis = 1),('Compliance BSC (Y/N)')] = 'Yes (FAR own trade)'


#to identify FAR own trade policy using client's email address
compliance.loc[compliance['All E-mail Addresses.1'].notna() & compliance['All E-mail Addresses.1'].str.contains('ippfa.com'),'Compliance BSC (Y/N)'] = "Yes (FAR own trade)"

#open the 'IPPFA approved list' for 'Term Life' insurance plan
pro = pro.reindex(['Plan Type', 'Product Classification', 'Type', 'Carrier', 'Plan Name',
       'LPHR / Approved'], axis = 'columns', fill_value = '')
pro.dropna(axis = 0, inplace = True)

#merge both files for easy data manipulation (to check whether there is wrong labelling for 'Term Life')
compliance = pd.merge(compliance, pro, how = 'left', on = 'Plan Name')

#to check against the IPPFA Approved list
compliance.rename(columns = {'Plan Type_x':'Plan Type'}, inplace = True)
compliance['Type'].replace(np.NaN, '', inplace = True)
compliance['Plan Type'].replace(np.NaN, '', inplace = True)
check_1 = compliance['Type'].str.contains('Term Life')
#check_2 = compliance['Plan Type_x'] != compliance['Type']
compliance.loc[check_1, 'Plan Type'] = compliance['Type']
compliance.loc[compliance['Plan Type'] == '', 'Plan Type'] = compliance['Type']

#for term life, when can't find plan in the approved list then "No"
#hi_1 = compliance['Plan Type'] == 'Term Life'
#hi_2 = compliance['Type'] == ''
#compliance.loc[hi_1 & hi_2, 'Compliance BSC (Y/N)'] = 'No'

#for term life, wrong labelling for 'Plan Type' in the compliance file
hi_1 = compliance['Plan Type'] == 'Term Life'
hi_2 = ~compliance['Type'].str.contains('Term Life')
hi_3 = compliance['Type'] != ''

compliance.loc[hi_1 & hi_2 & hi_3, 'Compliance BSC (Y/N)'] = 'No'


#inserting new columns
#compliance.insert(loc=21, column='Rider Remarks', value='')
import re

compliance.loc[compliance['Plan Type'].str.contains('Commercial|Dread Diseases/Critical Illness|Eldershield|Hospital & Surgical|Medishield', flags=re.IGNORECASE, regex = True),'Compliance BSC (Y/N)'] = 'No (A&H)'

compliance.loc[compliance['Plan Type'].str.contains('LTC'),'Compliance BSC (Y/N)'] = 'No (CIS)'

compliance.loc[compliance['Accredited Investor'].str.contains('Yes'),'Compliance BSC (Y/N)'] = 'No (Al)'

compliance.loc[compliance['Subject To BSC'].str.contains('Corporate') | compliance['Contact Name'].str.contains('pte|ltd', flags=re.IGNORECASE, regex = True) | compliance['Plan Name'].str.contains('HSBC', flags=re.IGNORECASE, regex = True),'Compliance BSC (Y/N)'] = 'No (Others)'

compliance.loc[compliance['Subject To BSC'] == '', 'Compliance BSC (Y/N)'] = 'No'

compliance.loc[compliance['Subject To BSC'].str.contains('A&H'),'Compliance BSC (Y/N)'] = 'No (A&H)'

hi_4 = compliance['Subject To BSC'] == ''

compliance.loc[hi_1 & hi_4,'Compliance BSC (Y/N)'] = 'Yes'

# create new columns with blank value in order to match the Ins file
#renaming columns to match Ins file
column_1 = ['Advisor',"All E-mail Addresses",'Created On',"Contact Name","Contact ID#","All E-mail Addresses.1","Complete Address","Unformatted Postal code","DOB","Age","Nationality","Job Title","Annualized Premium with Rider","Carrier Name","Policy #","Plan Name","Plan Type","Effective Date","Status","Status Date","Subject To BSC", "Compliance BSC (Y/N)","IPP Advisory Group","All Advisors","Modified On","Case Modified On","Accrediated Investor"]

compliance = compliance.reindex(column_1, axis = 'columns')


#Filter old advisor information
aacond = 'Old -'
check1 = (compliance['All Advisors'].str.contains(aacond)) & (compliance['Compliance BSC (Y/N)'] == 'Yes')
oldadvisordf = compliance[check1]
#oldadvisordf.to_excel(oldadvisor_file)

#Filter all Compliance BSC Yes information
condition1 = compliance['Compliance BSC (Y/N)'].str.contains('Yes')
compliance = compliance[condition1]

##to manage investment data
#calculate age
now = pd.Timestamp('now')
cis["Client's DOB"] = pd.to_datetime(cis["Client's DOB"], format = '%Y-%m-%d')
cis["Client's DOB"].head()
cis["Client's DOB"] = cis["Client's DOB"].where(cis["Client's DOB"] < now, cis["Client's DOB"] -  np.timedelta64(100, 'Y'))
cis['Age'] = (now - cis["Client's DOB"]).astype('<m8[Y]')

#to extract digit only for lump sum amount

import re

def extract_number(val: str):
    if not isinstance(val, str):
        return val
    regex = r"\D*(\d+)\D*"
    match = re.match(regex, val)
    return match.group(1)

cis['Lump Sum'] = cis['Lump Sum'].apply(extract_number)
cis['Lump Sum'] = pd.to_numeric(cis['Lump Sum'])

#replace RSP NaN to 0
index_rsp = cis.loc[cis['RSP'].isna(),'RSP'].index
cis.loc[cis['RSP'].isna(),'RSP'] = 0
cis.loc[cis['Lump Sum'].isna(), 'Lump Sum'] = 0

#create a new column converting frequency to number
cis['No Frequency (RSP ONLY)'] = cis['Frequency (RSP ONLY)'].apply(lambda x: 2 if x == 'Semi-annually' else (4 if x == 'Quarterly' else (12 if x == 'Monthly' else 0)))

#create RSP total using the new column value above
cis['RSP Total'] = cis['No Frequency (RSP ONLY)'] * cis['RSP']

#sum RSP and lump sum in a new column 'Annualized Premium with Rider'
cis['Annualized Premium with Rider'] = cis['Lump Sum'] + cis['RSP Total']
#cis['Annualized Premium with Rider'] = cis.loc[:,['Lump Sum','RSP Total']].sum(axis=1)

#create 'Final BSC (Y/N)' column
cis['Final BSC (Y/N)'] = cis['Remarks'].apply(lambda x: 'Yes (FAR own trade)' if x == 'FAR own case' else ('No Al' if x == 'Al' else 'Yes'))

#overwrite 'Final BSC (Y/N)' value to 'Yes (FAR own trade)' for both same email address and both same name
cis.loc[(cis["Advisor's email address"] == cis["Client's Email Add"]),'Final BSC (Y/N)'] = 'Yes (FAR own trade)'
cis.loc[(cis["Advisor"] == cis["Client's Name"]),'Final BSC (Y/N)'] = 'Yes (FAR own trade)'

#new columns 'Effective Date' & 'Status Date' same as 'Submission Date'
cis['Effective Date'] = cis['Submission Date']
cis['Status Date'] = cis['Submission Date']

# create new columns with blank value in order to match the Ins file
#renaming columns to match Ins file
cis.rename(columns = {'Transaction Type':'Plan Type'}, inplace = True)
column_1 = ['Advisor',"Advisor's email address",'Submission Date',"Client's Name","Client's NRIC","Client's Email Add","Complete Address","Unformatted Postal code","Client's DOB","Age","Nationality","Job Title","Annualized Premium with Rider","Carrier","Policy No.","Plan Name","Plan Type","Effective Date","Mode","Status Date","Subject To BSC","Final BSC (Y/N)","IPP Advisory Group","All Advisors","Modified On","Case Modified On","Accrediated Investor"]
#column_1 = ['Advisor']
cis = cis.reindex(column_1, axis = 'columns')

condition1 = cis['Final BSC (Y/N)'].str.contains('Yes')
cis = cis[condition1]


compliance.columns = cis.columns

#pd.concat([compliance,cis],ignore_index = False)
combined = pd.concat([compliance,cis],ignore_index = False, keys = ['Ins', 'Cis'], names = ['From where?'])
combined = combined.reset_index().drop(columns = 'level_1')
combined


#to check for similar name then group them as the same

for i, name in enumerate(combined['Advisor']):
    for _, lookup in enumerate(combined['Advisor']):
        if name in lookup: 
            combined['Advisor'][i] = lookup
            
            
#calculate the total number of policy for each advisor

combined['Total No Policy'] = ''

for i, name in combined['Advisor'].iteritems():
    combined_test1 = combined[combined['Advisor'] == name]
    combined['Total No Policy'][i] = len(combined_test1)
    
combined['Round 1 Audit'] = combined['Total No Policy'].apply(lambda x: max(1,round(x*0.05)))
combined['Round 2 Audit'] = combined['Total No Policy'].apply(lambda x: max(1,round(x*0.1)) if x > 1 else 0)
combined['Round 3 Audit'] = combined['Total No Policy'].apply(lambda x: max(1,round(x*0.2)) if x > 2 else 0)


#option 1: find the median number for each advisor then match against their cumulative count to extract the dataframe

import math

combined['Median'] = ''

for i, name in combined['Advisor'].iteritems():
    combined_test1 = combined[combined['Advisor'] == name]
    combined_test1_median_index = math.floor((len(combined_test1)+1)/2)
    combined['Median'][i] = combined_test1_median_index
    #median_test1 = combined_test1.iloc[combined_test1_median_index]

combined['Cumulative'] = combined.groupby('Advisor').cumcount() + 1


#option 2: find the median number for each advisor then match against their cumulative count to extract the dataframe
#round 1

median_row_all = []
result = []

for _, v in enumerate(combined['Advisor'].unique()):
    temp_df = combined[combined['Advisor'] == v]
    temp_df.reset_index(inplace = True)
    median_index = temp_df.at[0,'Median']
    round_no = temp_df.at[0,'Round 1 Audit']

    for j in range(round_no):
        median_row = median_index - 1 + j
        median_row_all.append(temp_df.iloc[[median_row]])
#    median_row_all.append(median_row)
#median_row = temp_df.iloc[[18,19]]
median_row_all = pd.concat(median_row_all)

#round 2
median_row_all_r2 = []
result = []

for _, v in enumerate(combined['Advisor'].unique()):
    temp_df = combined[combined['Advisor'] == v]
    temp_df.reset_index(inplace = True)
    median_index = temp_df.at[0,'Median']
    previous_round = temp_df.at[0,'Round 1 Audit']
    round_no = temp_df.at[0,'Round 2 Audit']
    
    for j in range(round_no):
        median_row = median_index + previous_round - 1 + j
        median_row_all_r2.append(temp_df.iloc[[median_row]])
#    median_row_all.append(median_row)
#median_row = temp_df.iloc[[18,19]]
median_row_all_r2 = pd.concat(median_row_all_r2)

#round 3
median_row_all_r3 = []
result = []

for _, v in enumerate(combined['Advisor'].unique()):
    temp_df = combined[combined['Advisor'] == v]
    temp_df.reset_index(inplace = True)
    median_index = temp_df.at[0,'Median']
    previous_round = temp_df.at[0,'Round 1 Audit'] + temp_df.at[0,'Round 2 Audit']
    round_no = temp_df.at[0,'Round 3 Audit']
    
    for j in range(round_no):
        median_row = median_index - 2 - j
        median_row_all_r3.append(temp_df.iloc[[median_row]])
#    median_row_all.append(median_row)
#median_row = temp_df.iloc[[18,19]]
median_row_all_r3 = pd.concat(median_row_all_r3)


#to check for similar name then group them as the same

for i, name in enumerate(combined['Advisor']):
    for _, lookup in enumerate(combined['Advisor']):
        if name in lookup: 
            combined['Advisor'][i] = lookup


#create a new dataframe for audit number purpose
advisor_count = combined.loc[:, 'Advisor'].value_counts()

advisor_count = advisor_count.to_frame()

advisor_count.rename(columns = {'Advisor': 'Total Number of Policy'}, inplace = True)
advisor_count['Round 1 Audit'] = advisor_count['Total Number of Policy'].apply(lambda x: max(1,round(x*0.05)))
advisor_count['Round 2 Audit'] = advisor_count['Total Number of Policy'].apply(lambda x: max(1,round(x*0.1)) if x > 1 else 0)
advisor_count['Round 3 Audit'] = advisor_count['Total Number of Policy'].apply(lambda x: max(1,round(x*0.2)) if x > 2 else 0)


#create a new dataframe for Advisory group

#to check for similar name then group them as the same

for i, name in enumerate(combined['IPP Advisory Group']):
    for _, lookup in enumerate(combined['IPP Advisory Group']):
        if name.casefold() in lookup.casefold(): 
            combined['IPP Advisory Group'][i] = lookup


advisory_group = combined.loc[:, 'IPP Advisory Group'].value_counts()

advisory_group = advisory_group.to_frame()

advisory_group.rename(columns = {'IPP Advisory Group': 'Total Number of Cases'}, inplace = True)
advisory_group['Round 1 Total Number of Cases'] = median_row_all.loc[:, 'IPP Advisory Group'].value_counts()
advisory_group['Round 2 Total Number of Cases'] = median_row_all_r2.loc[:, 'IPP Advisory Group'].value_counts()
advisory_group['Round 3 Total Number of Cases'] = median_row_all_r3.loc[:, 'IPP Advisory Group'].value_counts()


# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('test11.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
combined.to_excel(writer, sheet_name = 'Ins + Cis (Y)')
median_row_all.to_excel(writer, sheet_name = 'Round 1 Audit')
median_row_all_r2.to_excel(writer, sheet_name = 'Round 2 Audit')
median_row_all_r3.to_excel(writer, sheet_name = 'Round 3 Audit')
advisor_count.to_excel(writer, sheet_name = 'Advisor Counts')
advisory_group.to_excel(writer, sheet_name = 'Advisory Group Counts')


# Close the Pandas Excel writer and output the Excel file.
writer.save()